# STA 561 HW 10 {-}

Authors  
 -  Alonso Guerrero Castaneda (UID: 1194613)  
 -  Eli Gnesin (UID: 1172961)  
 -  Tommy Misikoff (UID: 1166813)  
 -  Sanskriti Purohit (UID: 1179957)  
 -  Will Tirone (UID: 1130904)  

TA: Rick Presman 

In [19]:
#Import Lines
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from scipy.spatial import ConvexHull

## Question 1 {-}

What is the optimal decision rule $\pi$?

In [5]:
n = 500
B = 1000
b0 = b1 = b2 = 1
b3 = 0.5
tau = [0.01, 0.025]
p_values = []

#Create our random number generator
rng = np.random.default_rng()

for t in tau:
    for b in range(B):
        # Create our X
        samp = rng.normal(loc = 0, scale = 1, size = n)
        X = np.power(samp, 5) #If X^{1/5} is N(0,1), then X is N(0,1)^5
        #print(X)

        #Create A
        A = [1 if np.abs(x) <= t else 0 for x in X]
        #print(A)

        #Create Y
        err = rng.normal(loc = 0, scale = 1, size = n)
        Y = (b0 * np.ones(n)) + (b1 * A) + (b2 * X) + (b3 * (A * X)) + err
        #print(Y)

        #Build a dataframe
        df = pd.DataFrame(list(zip(X, A, Y)))
        df.columns = ["X", "A", "Y"]

        #Now the OLS
        mod = sm.OLS.from_formula("Y ~ A + X + A*X", data = df)
        res = mod.fit()
        #print(res.pvalues[3]) #This is the p-value for beta_3
        p_values.append(res.pvalues[3])

    print(f"Tau = {t}")
    print(f"The proportion of significant betas was {np.mean([1 if x < 0.05 else 0 for x in p_values])}")

Tau = 0.01
The proportion of significant betas was 0.041
Tau = 0.025
The proportion of significant betas was 0.0445


## Question 2: Constructing Kernels

### Part 1

Steps: 

1. generate $Z_1,...Z_B$ from convex hull of support of $X_1,...,X_n$
    - is X random? 
2. create outputs $\hat{f_n}(Z_1),...,\hat{f_n}(Z_B)$
    - how do we create these? 
3. fit $\bigg\{ (Z_b, \hat{f_n}(Z_b)\bigg\}_{b=1}^{B}$

In [70]:
d = datasets.load_iris(return_X_y=False, as_frame=True)
X = d['data']
y = d['target']

hull = ConvexHull(X).simplices # returns uhh something 

In [74]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf_fit = clf.fit(X,y)

In [79]:
clf_fit.predict([[7,3,5,1.8]])

C:\Users\WillTirone\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([2])

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
# fit the local linear model 